### INMET INFO

In [2]:
import os
import pandas as pd
from ppt.config.database.tables import INMET
from ppt.config.database.connector import (generate_database_engine,
                                           generate_database_session)

# Conectando ao banco postgres 
session = generate_database_session()
conn = generate_database_engine()


2024-04-27 22:17:00,777 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-04-27 22:17:00,778 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-27 22:17:00,783 INFO sqlalchemy.engine.Engine select current_schema()
2024-04-27 22:17:00,785 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-27 22:17:00,789 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-04-27 22:17:00,790 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-27 22:17:00,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-27 22:17:00,805 INFO sqlalchemy.engine.Engine SELECT DISTINCT "INMET"."STATION", "INMET"."LATITUDE", "INMET"."LONGITUDE" 
FROM "INMET"
2024-04-27 22:17:00,806 INFO sqlalchemy.engine.Engine [generated in 0.00283s] {}
2024-04-27 22:17:02,918 INFO sqlalchemy.engine.Engine ROLLBACK


## Encontrando Estações Metereologicas
Para comparar a precipitação estimada por satélites com a precipitação real, primeiro precisamos extrair as coordenadas de todas as estações listadas na base de dados do INMET. Essas informações serão salvas em um arquivo Parquet. Em seguida, os dados dos satélites serão lidos e os valores correspondentes serão extraídos para outra tabela no banco de dados PostgreSQL. Neste caso, vou reduzir os pontos ecolhendo apenas estações dentro do estado de São Paulo.

In [3]:
# Importando estações e suas coordenadas
query = session.query(INMET.STATION, INMET.LATITUDE, INMET.LONGITUDE).distinct()
banco_inmet_unique = pd.read_sql_query(query.statement, conn)
print(banco_inmet_unique)

                     STATION   LATITUDE  LONGITUDE
0       FREDERICO WESTPHALEN -27.395556 -53.429444
1                   ALMENARA -16.166667 -40.687778
2          SAO JOAO DO PIAUI  -8.364444 -42.252500
3     SAO LUIZ DO PARAITINGA -23.228333 -45.416944
4                       UNAI -16.553889 -46.881944
...                      ...        ...        ...
1476               ARIQUEMES  -9.949001 -62.961873
1477                  JAPIRA -23.773341 -50.180570
1478                   AVARE -23.099722 -48.945556
1479        CAMPINA DA LAGOA -24.570861 -52.800256
1480                OIAPOQUE   3.813583 -51.862500

[1481 rows x 3 columns]


In [7]:
# Salvando todas as estações
#banco_inmet_unique.to_csv("Files/stations_inmet.csv")

## Extraindo apenas as estações de SP
Joguei os pontos disponiveis no banco para o arquivo stations_inmet.csv. Usando o QGIS fiz a interseção das estações dentro do estado de São Paulo.

In [19]:
# Pontos 
points = pd.read_parquet("Files/stations_inmet_sp.parquet")
points = points[['STATION', 'LATITUDE', 'LONGITUDE']]
stations = [row['STATION'] for index, row in points.iterrows()]
print(points, stations)

                    STATION   LATITUDE  LONGITUDE
0    SAO LUIZ DO PARAITINGA -23.228333 -45.416944
1                  IBITINGA -21.855567 -48.799717
2                  OURINHOS -22.948611 -49.894167
3                  BERTIOGA -23.845000 -46.143056
4               CASA BRANCA -21.779722 -47.079722
..                      ...        ...        ...
104          JOSE BONIFACIO -21.085556 -49.920556
105              PRADOPOLIS -21.338432 -48.114007
106                OURINHOS -23.000556 -49.843333
107  SAO PAULO - INTERLAGOS -23.724722 -46.678056
108                   AVARE -23.099722 -48.945556

[109 rows x 3 columns] ['SAO LUIZ DO PARAITINGA', 'IBITINGA', 'OURINHOS', 'BERTIOGA', 'CASA BRANCA', 'CAMPOS DO JORDAO', 'VALPARAISO', 'DRACENA', 'TAUBATE', 'PRADOPOLIS', 'BARUERI', 'SAO PAULO - MIRANTE', 'SAO MIGUEL ARCANJO', 'BRAGANCA PAULISTA', 'BARUERI', 'SAO MIGUEL ARCANJO', 'VOTUPORANGA', 'JALES', 'CAMPOS DO JORDAO', 'PARANAPOEMA', 'MARILIA', 'SAO PAULO - MIRANTE', 'ITUVERAVA', 'IGUAPE', 'V

In [ ]:
# Lista para armazenar os dados de todas as estações
dados_estacoes = []

# Iterar sobre as estações
for station in stations:
    query_sp = session.query(INMET).filter(INMET.STATION == station)
    dados_estacao = pd.read_sql_query(query_sp.statement, conn)
    dados_estacoes.append(dados_estacao)

# Concatenar todos os DataFrames em um único DataFrame
banco_inmet_unique = pd.concat(dados_estacoes, ignore_index=True)


In [26]:
print(banco_inmet_unique.head())

              KEY       DATE  CODE   LATITUDE  LONGITUDE  \
0  2022-01-01A740 2022-01-01  A740 -23.228333 -45.416944   
1  2022-01-02A740 2022-01-02  A740 -23.228333 -45.416944   
2  2022-01-03A740 2022-01-03  A740 -23.228333 -45.416944   
3  2022-01-04A740 2022-01-04  A740 -23.228333 -45.416944   
4  2022-01-05A740 2022-01-05  A740 -23.228333 -45.416944   

                  STATION  UF  PPT_mm  
0  SAO LUIZ DO PARAITINGA  SP     0.0  
1  SAO LUIZ DO PARAITINGA  SP     0.0  
2  SAO LUIZ DO PARAITINGA  SP     0.0  
3  SAO LUIZ DO PARAITINGA  SP     0.0  
4  SAO LUIZ DO PARAITINGA  SP     0.0  
